# 일단 읽어보기

In [1]:
import pandas as pd
import numpy as np

datapath = './data/arcene_train.data'
data1 = pd.read_csv(datapath, sep=' ')

data1.shape

(99, 10001)

In [2]:
data1.head()

,0,71,0.1,95,0.2,538,404,20,0.3,0.4,...,570.9,86.12,0.5762,36.31,0.5763,80.18,0.5764,0.5765,524,Unnamed: 10000
0,0,41,82,165,60,554,379,0,71,0,...,605,69,7,473,0,57,0,284,423,NaN
1,0,0,1,40,0,451,402,0,0,0,...,593,28,0,24,0,90,0,34,508,NaN
2,0,56,44,275,14,511,470,0,0,0,...,600,0,26,86,0,102,0,0,469,NaN
3,105,0,141,348,0,268,329,0,0,1,...,0,0,0,0,190,301,0,0,354,NaN
4,38,62,0,251,75,515,0,9,85,300,...,454,0,36,2,369,194,18,59,340,NaN


In [4]:
data1 = pd.read_csv(datapath, sep=' ', names =['f{}'.format(x) for x in range(0,10001)])

del(data1['f10000'])
data1.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f9990,f9991,f9992,f9993,f9994,f9995,f9996,f9997,f9998,f9999
0,0,71,0,95,0,538,404,20,0,0,...,255,570,86,0,36,0,80,0,0,524
1,0,41,82,165,60,554,379,0,71,0,...,213,605,69,7,473,0,57,0,284,423
2,0,0,1,40,0,451,402,0,0,0,...,235,593,28,0,24,0,90,0,34,508
3,0,56,44,275,14,511,470,0,0,0,...,91,600,0,26,86,0,102,0,0,469
4,105,0,141,348,0,268,329,0,0,1,...,813,0,0,0,0,190,301,0,0,354


In [5]:
data1.shape

(100, 10000)

# 모든 데이터 읽기

In [24]:
def read_data_file(filepath):
    data = pd.read_csv(filepath, sep=' ', names =['f{}'.format(x) for x in range(0,10001)])

    del(data['f10000'])
    return data

def read_label_file(filepath):
    labels = pd.read_csv(filepath, sep=' ', names =['class'])
    return labels

data1 = read_data_file('./data/arcene_train.data')
data2 = read_data_file('./data/arcene_valid.data')

label1 = read_label_file('./data/arcene_train.labels')
label2 = read_label_file('./data/arcene_valid.labels')

merged_data = pd.concat([data1, data2], ignore_index = True)
merged_label = pd.concat([label1, label2], ignore_index = True)

print(merged_data.shape, merged_label.shape)

(200, 10000) (200, 1)


In [25]:
final_data = pd.concat([merged_data, merged_label], axis = 1)
print(final_data.shape)
final_data.head()

(200, 10001)


,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f9991,f9992,f9993,f9994,f9995,f9996,f9997,f9998,f9999,class
0,0,71,0,95,0,538,404,20,0,0,...,570,86,0,36,0,80,0,0,524,1
1,0,41,82,165,60,554,379,0,71,0,...,605,69,7,473,0,57,0,284,423,-1
2,0,0,1,40,0,451,402,0,0,0,...,593,28,0,24,0,90,0,34,508,1
3,0,56,44,275,14,511,470,0,0,0,...,600,0,26,86,0,102,0,0,469,1
4,105,0,141,348,0,268,329,0,0,1,...,0,0,0,0,190,301,0,0,354,-1


In [26]:
# class를 0과 1로 바꿔주기
from sklearn import preprocessing

le_class = preprocessing.LabelEncoder()
final_data['class'] = le_class.fit_transform(final_data['class'])

final_data.tail()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f9991,f9992,f9993,f9994,f9995,f9996,f9997,f9998,f9999,class
195,24,73,0,436,92,400,0,0,139,261,...,540,0,86,130,365,58,17,3,37,0
196,11,58,50,332,109,393,122,0,75,134,...,355,156,77,26,277,265,0,36,261,0
197,93,32,137,319,0,264,231,21,0,0,...,9,0,0,0,244,309,0,276,312,1
198,119,12,198,339,0,289,410,0,0,4,...,0,37,0,0,256,402,0,0,350,1
199,112,19,171,334,0,282,208,0,0,0,...,0,118,0,0,226,379,0,0,367,0


# 문제 정의 및 해결방법 결정

In [37]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

features = ['f{}'.format(x) for x in range(0,10000)]

kf = KFold(n_splits = 10, shuffle = True)

accrs = []
fold_idx = 1

for train_idx, test_idx in kf.split(final_data):
    print('fold {}'.format(fold_idx))
    train_d, test_d = final_data.iloc[train_idx], final_data.iloc[test_idx]
    
    train_y = train_d['class']
    train_x = train_d[features]
    
    test_y = test_d['class']
    test_x = test_d[features]
    
    model = LogisticRegression(solver = 'lbfgs', max_iter = 300) # solver의 파라미터 lbfgs # max_iter 기본값 100
    model.fit(train_x, train_y)
    
    mean_accr = model.score(test_x, test_y)
    accrs.append(mean_accr)
    # print(mean_accr)
    fold_idx +=1
    
print(np.average(accrs))

fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9
fold 10
0.9099999999999999


# PCA 적용

In [40]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

features = ['f{}'.format(x) for x in range(0,10000)]

kf = KFold(n_splits = 10, shuffle = True, random_state = 100)

accrs = []
fold_idx = 1

for train_idx, test_idx in kf.split(final_data):
    print('fold {}'.format(fold_idx))
    train_d, test_d = final_data.iloc[train_idx], final_data.iloc[test_idx]
    
    pca = PCA(n_components = 100) # 1로 줄이면 성능 하락 
    
    train_y = train_d['class']
    train_x = pca.fit_transform(train_d[features])
    
    print(train_x.shape)
    
    test_y = test_d['class']
    test_x = pca.transform(test_d[features]) # 테스트 데이터에 대한 fit_transform이 아닌 transform
    
    model = LogisticRegression(solver = 'lbfgs', max_iter = 300) # solver의 파라미터 lbfgs # max_iter 기본값 100
    model.fit(train_x, train_y)
    
    mean_accr = model.score(test_x, test_y)
    accrs.append(mean_accr)
    # print(mean_accr)
    fold_idx +=1
    
print(np.average(accrs))

fold 1
(180, 100)
fold 2
(180, 100)
fold 3
(180, 100)
fold 4
(180, 100)
fold 5
(180, 100)
fold 6
(180, 100)
fold 7
(180, 100)
fold 8
(180, 100)
fold 9
(180, 100)
fold 10
(180, 100)
0.8800000000000001


# LDA

In [42]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

features = ['f{}'.format(x) for x in range(0,10000)]

kf = KFold(n_splits = 10, shuffle = True, random_state = 100)

accrs = []
fold_idx = 1

for train_idx, test_idx in kf.split(final_data):
    print('fold {}'.format(fold_idx))
    train_d, test_d = final_data.iloc[train_idx], final_data.iloc[test_idx]
    
    # LDA는 축 개수는 최대 class -1까지 가능
    lda = LinearDiscriminantAnalysis(n_components = 1)
    
    train_y = train_d['class']
    train_x = lda.fit_transform(train_d[features],train_y)
    
    print(train_x.shape)
    
    test_y = test_d['class']
    test_x = lda.transform(test_d[features]) # 테스트 데이터에 대한 fit_transform이 아닌 transform
    
    model = LogisticRegression(solver = 'lbfgs', max_iter = 300) # solver의 파라미터 lbfgs # max_iter 기본값 100
    model.fit(train_x, train_y)
    
    mean_accr = model.score(test_x, test_y)
    accrs.append(mean_accr)
    # print(mean_accr)
    fold_idx +=1
    
print(np.average(accrs))

fold 1


/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


(180, 1)
fold 2


/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


(180, 1)
fold 3


...

/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


(180, 1)
fold 9


/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


(180, 1)
fold 10
(180, 1)
0.795


/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
